## Quickstart
This document provides a quick demonstration of the basic usage of tagreader. It will show you the steps from importing the package to fetching data and making a plot. Some cells contain links to more details that can be found in the [manual](./manual.ipynb).

In [1]:
# Initialization code needed for this notebook. No need to enter this yourself.
import sys
sys.path.insert(0, "../")

Start by importing the package:

In [2]:
import tagreader

If we don't know the name of the data source, we can check which PI and IP.21 servers we have access to ([more details](./manual.md#list-available-resources)):

In [3]:
print(tagreader.list_pi_sources())

['ims-pors-01', 'JSV', 'ONO-IMS', 'PIMAM', 'PINO', 'PISFA.equinor.com', 'PISFB.equinor.com', 'PISFC.equinor.com', 'SLP']


Similarly we could have typed `tagreader.list_aspen_servers()` to get a list of available Aspen InfoPlus.21 servers.

Let's define a connection to PINO. We need to specify that PINO is a PI server. Available types are `pi`, `ip21` `piwebapi` or `aspenone`, all case-insensitive.

In [4]:
c = tagreader.IMSClient('PINO', 'pi')

By default a cache-file using the HDF5 file format will be created to store previously read data values. The cache can speed up subsequent reads by a decade or two, and is therefore highly recommended. If, for any reason, you want to disable the cache, set it to `None` before making the connection:

In [5]:
c.cache = None

We can now establish a connection to the server:

In [6]:
c.connect()

After connecting, we can search for a tag. `%` and `*` are valid and equivalent wildcards:

In [7]:
print(c.search('cd*158'))

[('CDEP158', 'Light Naphtha End Point'), ('CDM158', 'Light Naphtha End Point Control'), ('CDT158', 'Atmospheric Tower OH Vapor')]


We can also search for a description:

In [8]:
print(c.search(desc='*reactor*'))

AttributeError: 'IMSClient' object has no attribute 'search_tag'

Or both tag and description, which returns those that fulfill both requirements:

In [ ]:
print(c.search(tag='BA:*', desc='*Temperature*'))

Once we know the tag name, we can read values for a duration of 3.5 hours starting January 5th at 8 in the morning with 3-minute (180-seconds) intervals. The default query method is interpolated. Timestamps are parsed using [pandas.Timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html), and can therefore be provided in many different string formats.

In [ ]:
df = c.read(['BA:ACTIVE.1', 'BA:LEVEL.1', 'BA:CONC.1'], '05-Jan-2020 08:00:00', '05/01/20 11:30am', 180)

*Note*: Tags with maps (relevant for some InfoPlus.21 servers) can be specified on the form `'tag;map'`, e.g. `'17B-HIC192;CS A_AUTO'`.

The result from the last call is a Pandas dataframe, and can be manipulated as such:

In [ ]:
df.tail()

In [ ]:
df['BA:LEVEL.1'].size

In [ ]:
df['BA:CONC.1'].loc['2020-01-05 11:24:00']

In [ ]:
max(df['BA:LEVEL.1'])

etc...

Sometimes it can be handy to obtain the unit and description for the three tags:

In [ ]:
tags = ['BA:ACTIVE.1', 'BA:LEVEL.1', 'BA:CONC.1']
units = c.get_units(tags)
desc = c.get_descriptions(tags)
print(units)
print(desc)

In [ ]:
tag = 'BA:CONC.1'
df[tag].plot(grid=True, title=desc[tag]).set_ylabel(units[tag])